In [1]:
import pandas as pd
import numpy as np
import pickle
import os

In [6]:
folder_to_summarize = "hard"

dataframe = pd.DataFrame(columns=["source_query", "text"])
for filename in os.listdir(folder_to_summarize):
    source_query = " ".join(filename.split("_")[3:]).replace(".pkl", "")
    if filename.endswith(".pkl"):
        with open(os.path.join(folder_to_summarize, filename), 'rb') as f:
            data = pickle.load(f)
        data_types = ["publications", "resources", "projects","others"]
        texts = []
        for data_type in data_types:
            for data_object in data[data_type]:
                text = ""
                if hasattr(data_object, "name") and len(data_object.name)>0:
                    text = data_object.name
                if hasattr(data_object, "description") and len(data_object.description)>0:
                    text += "\n"+ data_object.description
                elif hasattr(data_object, "abstract") and len(data_object.abstract)>0:
                    text += "\n"+ data_object.abstract
                if hasattr(data_object, "keywords") and len(data_object.keywords)>0:
                    text += "\nKeywords:"+ ", ".join(data_object.keywords)
                texts.append(text)
        query_texts = pd.DataFrame({"source_query": [source_query]*len(texts), "text": texts})
        dataframe = pd.concat([dataframe, query_texts], ignore_index=True)

shuffled_dataframe = dataframe.sample(frac=1).reset_index(drop=True)
# Save the dataframe to pickle
savepath = f"{folder_to_summarize}_combined_dataframe.pkl"
with open(savepath, 'wb') as f:
    pickle.dump(shuffled_dataframe, f)